In [4]:
import os
os.environ['GROQ_API_KEY'] = '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'


In [8]:
import requests
import json
import pandas as pd
import groq
import os
import time
from tqdm import tqdm

# API Client for Groq
client = groq.Groq(
    api_key=os.environ.get('GROQ_API_KEY')  # Make sure this is set
)

# Load your data
df = pd.read_csv('/content/MLSentiment.csv')

# Combine title and text
df['combined'] = df['title'].fillna('') + " " + df['text'].fillna('')

# Stance categories
stance_categories = ["Oppose", "Neutral", "Approve", "Irrelevant"]

# Prompt creator
def create_prompt(post_text):
    return (
        f"Given the following Reddit post, classify the stance in response to the question: "
        f"'How did Reddit portray the US in Ukraine before and after the Trump-Zelensky White House Confrontation?'\n\n"
        f"Post: {post_text}\n\n"
        f"Answer the stance in one word: {', '.join(stance_categories)}"
    )

# LLM prediction function
def get_stance_prediction(post_text):
    time.sleep(1)  # Optional: prevent rate-limiting
    prompt = create_prompt(post_text)

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        model="gemma2-9b-it"
    )

    return chat_completion.choices[0].message.content.strip()

# Apply prediction with progress bar
tqdm.pandas(desc="Classifying stances")
df['stance_predictions'] = df['combined'].progress_apply(get_stance_prediction)

# Final output
print(df[['title', 'text', 'stance_predictions']])


Classifying stances: 100%|██████████| 310/310 [59:02<00:00, 11.43s/it]

                                                 title text  \
0    Tens of thousands of anti-government protester...  NaN   
1     This is Poland film about Poland from year 1940.  NaN   
2    Rally in Srpska: We came to defend the preside...  NaN   
3    The grants and loans given to Ukraine by diffe...  NaN   
4    You cant insult the country that protects yo...  NaN   
..                                                 ...  ...   
305  RU POV: Drone ignored civilians and seeked mil...  NaN   
306  RU POV: President Putin "The European politici...  NaN   
307  Ru POV: S-300 gets destroyed by a Iskander mis...  NaN   
308  Ru pov:A bunch of photos of a Russian soldier ...  NaN   
309  Ru pov:footage of ammunition delivery to the f...  NaN   

                                    stance_predictions  
0                                           Irrelevant  
1                                           Irrelevant  
2                                           Irrelevant  
3    Irrelevant

In [9]:
df.to_csv('LLM_Stance.csv', index=False)